In [1]:
import matplotlib.pyplot # Globalne ustawienia wykresów
matplotlib.pyplot.rcParams['axes.spines.top'] = False
matplotlib.pyplot.rcParams['axes.spines.right'] = False
matplotlib.pyplot.rcParams['axes.titlelocation'] = 'left'
matplotlib.pyplot.rcParams['axes.titleweight'] = 'bold'
matplotlib.pyplot.rcParams['figure.autolayout'] = True
matplotlib.pyplot.rcParams['figure.figsize'] = (15, 6)
from matplotlib.pyplot import subplots # 🎨 Wykresy
from IPython.display import YouTubeVideo # Wideo YT

In [2]:
# 🎵 Klasa sygnału do zebrania ważnych funkcji w całość
from pandas import Series # padnas do przechowywania danych
class Signal(): # Klasa sygnału 🎵 - ścieżka i jej metadane

    def __init__(self, series:Series, sample_rate:int):

        from scipy.interpolate import interp1d

        self.series, self.sample_rate = series, sample_rate
        self.interpolate = interp1d(series.index, series.values, fill_value='extrapolate')
        self.interpolate:callable

    @classmethod
    def read(cls, f:str): # 💾 Wczytywanie pliku

        import librosa, numpy as np
        from pandas import Series

        series, sample_rate = librosa.load(f, sr=None)
        idx = np.arange(len(series)) / sample_rate
        
        series = Series(series, index=idx)
        return cls(series, sample_rate)
    
    # 🎨 Wykres spektrogramu sygnału w danym zakresie 
    def spectrogram(self, range=None, ax=None, **kwargs):

        import matplotlib.pyplot

        if ax is None: ax = matplotlib.pyplot.gca()
        if range is None: range = (0, self.series.index[-1])

        ax.specgram(self.series.values, Fs=self.sample_rate, xextent=range, **kwargs)
        ax.set_xlabel('Czas [s]')
        ax.set_ylabel('Częstotliwość [Hz]')

    # 🎨 Wykres liniowy sygnału w domenie czasu
    def line(self, range=None, ax=None, **kwargs):

        import matplotlib.pyplot

        if ax is None: ax = matplotlib.pyplot.gca()
        if range is None: range = (0, self.series.index[-1])

        self.series.loc[range[0]:range[1]].plot(xlabel='Czas (s)', ylabel='Amplituda', ax=ax, **kwargs)

    # 🎨 Wykres FFT amplituda ~ częstotliwość
    def DFT(self, freq_range=None, ax=None, norm=False, **kwargs):

        from numpy import fft as Fourier, abs
        import matplotlib.pyplot

        if ax is None: ax = matplotlib.pyplot.gca()
        if freq_range is None: freq_range = (0, self.sample_rate/2)

        fft_values = Fourier.fft(self.series.values)
        fft_freq = Fourier.fftfreq(len(self.series.values), 1.0/self.sample_rate)

        if norm:
            fft_values = abs(fft_values) / len(fft_values)

        ax.plot(fft_freq, abs(fft_values), **kwargs)
        ax.set_xlim(freq_range)
        ax.set_xlabel('Częstotliwość [Hz]')
        ax.set_ylabel('Amplituda [DFT]')

    # 🎨 Wykres faza ~ częstotliwość
    def phase(self, freq_range=None, ax=None, **kwargs):

        from numpy import fft as Fourier, angle
        import matplotlib.pyplot

        if ax is None: ax = matplotlib.pyplot.gca()
        if freq_range is None: freq_range = (0, self.sample_rate/2)

        fft_values = Fourier.fft(self.series.values)
        fft_freq = Fourier.fftfreq(len(self.series.values), 1.0/self.sample_rate)

        ax.plot(fft_freq, angle(fft_values), **kwargs)
        ax.set_xlim(freq_range)
        ax.set_xlabel('Częstotliwość [Hz]')
        ax.set_ylabel('Faza [rad]')

    # 🎨 Wykresy złączone w jedną całość w domenie czasowej
    def time_stack(self, range=None, axs=None, kwargs:list[dict] = [{}]):

        import matplotlib.pyplot

        if axs is None: fig, axs = matplotlib.pyplot.subplots(2,1, sharex=True)
        if range is None: range = (0, self.series.index[-1])

        self.line(range, ax=axs[0], **kwargs[0] if len(kwargs) > 0 else {})
        self.spectrogram(range, ax=axs[1], **kwargs[1] if len(kwargs) > 1 else {})

    # 🎨 Wykresy złączone w jedną całość w dziedzinie częstotliwości
    def freq_stack(self, freq_range=None, axs=None, kwargs:list[dict] = [{}]):

        import matplotlib.pyplot

        if axs is None: fig, axs = matplotlib.pyplot.subplots(2,1, sharex=True)
        if freq_range is None: freq_range = (0, self.sample_rate/2)

        self.DFT(freq_range, ax=axs[0], **kwargs[0] if len(kwargs) > 0 else {})
        self.phase(freq_range, ax=axs[1], **kwargs[1] if len(kwargs) > 1 else {})

    # 🎨 Wykresy złączone w jedną całość w obu dziedzinach
    def stack(self, range=None, freq_range=None, axs=None, kwargs:list[dict] = [{}]):

        import matplotlib.pyplot

        if axs is None: fig, axs = matplotlib.pyplot.subplots(2,2, sharex='col')

        self.time_stack(range, axs=[axs[0,0], axs[1,0]], kwargs=kwargs[:2])
        self.freq_stack(freq_range, axs=[axs[0,1], axs[1,1]], kwargs=kwargs[2:])
        
        self.DFT(ax=axs[0,1], **kwargs[2] if len(kwargs) > 2 else {})
        
        for ax in [axs[0,1], axs[1,1]]: 
            ax.yaxis.set_label_position("right")
            ax.yaxis.tick_right()

    def apply(self, func:callable, **args):

        from pandas import Series

        series = self.series.copy()
        values = func(series.values, **args)

        series = Series(values, index=self.series.index)
        return Signal(series, self.sample_rate)
    
    def resample_with_rate(self, rate:int):

        from numpy import linspace
        from pandas import Series

        self.interpolate: callable
        self.sample_rate: int

        start = self.series.index[0]
        end = self.series.index[-1]
        length = end - start
        n_samples = round(length * rate)

        index = linspace(start, end, n_samples)
        series = Series(self.interpolate(index), index=index)

        return Signal(series, rate)

    def widget(self): # 🎛️ Widget do odtwarzania sygnału w notatniku Jupyter

        from IPython.display import Audio

        return Audio(self.series.values, rate=self.sample_rate)
    
def calc_alias(samp_freq, base_freq, num):
    
    alias = []
    for n in range(1, num + 1):
        
        alias_f = abs(base_freq - n * samp_freq)
        alias.append(alias_f)

    return alias

## Sygnał analogowy

Sygnały w rzeczywistości są zmianami w czasie i odbywają się sposób ciągły. Matematycznie można je zamodelować jako funkcje ciągłe. Co za tym idzie w każdym momencie ma jakąś wartość. Komputery z kolei mogą zapisywać jedynie skończoną ilość danych, co wynika z ich architektury bitowej. Co za tym idzie zbieranie danych o sygnałach wymaga ich **próbkowania**: wartość funkcji jest pobierana co pewien okres, najczęściej niewielki w porównaniu z okresem sygnału. W ten sposób sygnał ciągły jest zamieniany na sygnał dyskretny - na wykresie z linii ciągłej zostaje ciąg punktów.

Punkty zostają **zkwantyzowane** - przypisywana im jest najbliższa wartość, która może zostać zapisana w postaci cyfrowej.

Na końcu są **kodowane** do postaci binarnej.

**Twierdzenie o próbkowaniu** mówi, że próbkować trzeba z częstotliwością co najmniej dwukrotnie większą niż najwyższa częstotliwość sygnału. W przeciwnym wypadku sygnał zostanie zniekształcony, co nazywane jest efektem aliasingu.

**Aliasy**

Powstają gdy częstotliwość próbkowania jest za niska. Wtedy sygnał jest zniekształcony, bo nie jest w stanie zarejestrować wszystkich zmian sygnału.

$$
f_{\text{alias}} = 
    f_{0} - 
    \underset{k \in \mathbb{N}}{k}
    \cdot 
    f_{\text{próbkowanie}}
$$

**Filtr cyfrowy**:

* funkcja przekształcająca sygnał **cyfrowy na wejściu** w sygnał **cyfrowy na wyjściu**;
* filtry mogą być zbiorem **algorytmów na procesorze** albo **układem scalonym**;


**Filtr (cyfrowy) wygładzający**:
* modyfikuje składowe sygnału, żeby zredukować fluktuacje i zakłócenia;
* sygnał wynikowy jest bardziej regularny i spłaszczony w domenie czasu i/lub częstotliwości;

**Filtry FIR** (Finite Impulse Response):

* [Schemat działania](https://www.youtube.com/watch?v=NvRKtdrssFA) równoznaczy ze splotem sygnału i filtra `x[n] * filter`:
    1. `FFT(x[n])` - transformata Fouriera sygnału wejściowego
    2. `FFT(x[n]) · filter (F)` - przemnożenie w dziedzinie częstotliwościowej przez charakterystykę amplitudową filtra
    3. `IFFT(FFT(x[n]) · filter(F))` - transformata odwrotna: 
* rząd - liczba próbek w odpowiedzi impulsowej;
* metoda [okienkowego projektowania](https://www.youtube.com/watch?v=2tmilbi4L0o) filtrów cyfrowych
    * długość okna - liczba próbek w odpowiedzi impulsowej;
    * okno może mieć różne kształty;
    * filter praktyczny to splot idealnego filtra z oknem;
    * filtr praktyczny 
    * odchylenie amplitudy sygnału filtru - to te poboczne fale
    * obszar główny to obszar w którym amplituda jest największa - ma najwięcej energii - zależy od kształtu i długości okna
    * obszary boczne to pozostałe fragmetny fali - zależy tylko od kształtu okna

**Filtr Savitzkiego-Golaya**

* wygładza sygnał poprzez **dopasowanie wielomianu** wskazanego **stopnia**, na **oknie** wskazanej długości: linia wielomianu jest dopasowywana do punktów, a **centralny** punkt jest zastępowany wartością wielomianu w tym punkcie, iteracyjnie idzie po kolejnych punktach;
* redukuje zakłócenia w sygnale przy zachowaniu jego charakterystyki;
* filtr może być nieskóteczny przy sygnale o dużej zmienności;
* jest wrażliwy na parametry.

**Filtr uśredniająco wykładniczy**:

* Uśrednia wartości sygnału w oknie o **wykładniczo rosnącej** długości, co pozwala na **wygładzenie** sygnału z zachowaniem **informacji o trendzie**.